In [25]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import time
from datetime import datetime,timedelta
import schedule
import locale
import os
locale.setlocale(locale.LC_TIME, "id_ID.utf8")

'id_ID.utf8'

In [5]:
import schedule

In [14]:
def get_article_link(url):
    response = requests.get(url)
    page_content = html.fromstring(response.text)
    articles_link = page_content.xpath("//div[@class='artItem']//div[@class='artContent']/a[@class='artLink']/@href")
    return articles_link

def scrape_article(url):
    response = requests.get(url)
    page_content = html.fromstring(response.text)
    link = url

    title = page_content.xpath("//div[contains(@class,'detailsTitle')]/h1/text()")
    title = title[0].strip() if title else None

    date = None
    date_raw = page_content.xpath("//div[@class='detailsAttributeDates']/text()")
    if date_raw:
        candidate = date_raw[0].strip()
    else:
        date_alt = page_content.xpath("//span[@class='detailsAttributeItem']//text()")
        candidate = date_alt[-1].strip() if date_alt else None

    if candidate:
        for fmt in ['%A, %d %B %Y | %H:%M', '%A, %d %B %Y | %H:%M WIB']:
            try:
                date = datetime.strptime(candidate, fmt).isoformat()
                break
            except ValueError:
                continue
        if not date:
            print(f"[WARNING] Format tanggal tidak dikenali: {candidate} ({url})")
            date = candidate 

    full_text = page_content.xpath("//article[contains(@class,'detailsContent')]//p//text()")
    content = " ".join(p.strip() for p in full_text)

    return {"Link": link, "Judul": title, "Tanggal": date, "Isi": content}

def backtrack_crawlscrape(start_date,end_date):
    articles_data = []
    start_date = datetime.strptime(start_date,"%d-%m-%Y")
    end_date = datetime.strptime(end_date,"%d-%m-%Y")
    current = start_date
    
    base_url = "https://www.bisnis.com/index?date="
    articles_link = []
    while current <= end_date:
        url = f"https://www.bisnis.com/index?date={current}"
        print("Crawling:", url)
        articles_link.extend(get_article_link(url))
        current += timedelta(days=1)
        
    articles_link = list(dict.fromkeys(articles_link))
    
    for link in articles_link:
        print("Link:",link)
        articles_data.append(scrape_article(link))
    
    return articles_data

In [15]:
backtrack_crawlscrape("17-09-2025","17-09-2025")

Crawling: https://www.bisnis.com/index?date=2025-09-17 00:00:00
Link: https://premium.bisnis.com/read/20250917/658/1912354/danantara-dengan-berbagai-uluran-tangan-selamatkan-garuda-indonesia-giaa
Link: https://kabar24.bisnis.com/read/20250917/15/1912338/yusril-prabowo-tegaskan-tak-perlu-tim-pencari-fakta-untuk-usut-kerusuhan-demo
Link: https://kabar24.bisnis.com/read/20250917/19/1912353/israel-buka-jalur-evakuasi-tambahan-warga-gaza-tetap-menolak-pindah
Link: https://finansial.bisnis.com/read/20250917/215/1912326/bos-prudential-indonesia-ungkap-4-jurus-lawan-inflasi-medis
Link: https://ekonomi.bisnis.com/read/20250917/9/1912352/bi-optimistis-ekonomi-bakal-terkerek-di-semester-22025
Link: https://ekonomi.bisnis.com/read/20250917/98/1912337/4-urgensi-indonesia-butuh-ruu-pengelolaan-ruang-udara
Link: https://semarang.bisnis.com/read/20250917/536/1912344/bpd-diy-raih-dua-penghargaan-bisnis-indonesia-financial-award-2025
Link: https://ekonomi.bisnis.com/read/20250917/45/1912327/maruarar-sir

[{'Link': 'https://premium.bisnis.com/read/20250917/658/1912354/danantara-dengan-berbagai-uluran-tangan-selamatkan-garuda-indonesia-giaa',
  'Judul': 'Danantara dengan Berbagai Uluran Tangan Selamatkan Garuda Indonesia (GIAA)',
  'Tanggal': None,
  'Isi': 'Bisnis.com, JAKARTA – Badan Pengelola Investasi Danantara dengan berbagai uluran tangan turut serta dalam memperbaiki kinerja PT Garuda Indonesia (Persero) Tbk. (GIAA). Yang teranyar, Danantara terlibat langsung dan memegang komando dalam proses merger anak usaha PT Pertamina (Persero) yakni PT Pelita Air Service dengan Garuda Indonesia.'},
 {'Link': 'https://kabar24.bisnis.com/read/20250917/15/1912338/yusril-prabowo-tegaskan-tak-perlu-tim-pencari-fakta-untuk-usut-kerusuhan-demo',
  'Judul': 'Yusril: Prabowo Tegaskan Tak Perlu Tim Pencari Fakta untuk Usut Kerusuhan Demo',
  'Tanggal': '2025-09-17T23:50:00',
  'Isi': 'Bisnis.com, JAKARTA – Menteri Koordinator Bidang Hukum, HAM, Imigrasi, dan Pemasyarakatan Yusril Ihza Mahendra memasti

In [67]:
def standard_crawlscrape(path = "new_article.json"):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                data = []
    else:
        data = []
    
    existing_link = {a.get("Link") for a in data if "Link" in a}
    now_date = datetime.now().strftime("%Y-%m-%d %H:%M")
    print(now_date)
    url = f"https://www.bisnis.com/index?date={now_date}"
    
    count = 0
    links = get_article_link(url)
    for link in links:
        if link not in existing_link:
            print("Artikel baru:",link)
            article = scrape_article(link)
            data.append(article)
            existing_link.add(link)
            count+=1
    if count == 0:
        print("--Tidak ada artikel terbaru--")
    
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [68]:
standard_crawlscrape()

2025-09-18 06:47
--Tidak ada artikel terbaru--


In [ ]:
def job():
    print("Scrape artikel terbaru")
    standard_crawlscrape()
    
schedule.clear()
schedule.every(2).minutes.do(job)
while True:
    schedule.run_pending()
    time.sleep(1)

Scrape artikel terbaru
2025-09-18 06:47
--Tidak ada artikel terbaru--
Scrape artikel terbaru
2025-09-18 06:47
--Tidak ada artikel terbaru--


KeyboardInterrupt: 